# Data Wrangling

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# Using SDK targeting 2024-02-29-preview or 2023-10-31-preview, make sure your resource is in one of these regions: East US, West US2, West Europe
!pip install uv
!uv pip install --system langchain langchain-community docling natsort
from google.colab import userdata

Using Python 3.11.11 environment at: /usr
Resolved 129 packages in 413ms
Uninstalled 1 package in 20ms
Installed 1 package in 72ms
 - langchain-community==0.0.38
 + langchain-community==0.3.20


In [ ]:
# from docling.document_converter import DocumentConverter

# source = "/content/splitpdf/wren_and_martin_simplified_textbook-46.pdf"  # document per local path or URL
# converter = DocumentConverter()
# result = converter.convert(source)
# with open("output.md", "w") as f:
#   f.write(result.document.export_to_markdown())

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
from docling.document_converter import DocumentConverter
from natsort import natsorted
from multiprocessing import get_context
from concurrent.futures import ProcessPoolExecutor
import os

pdf_folder = "/content/splitpdf"
filelist = os.listdir(pdf_folder)
filelist = natsorted(filelist)
all_markdown_content = ""

for filename in filelist:
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)

        # Use Doclng to convert the PDF to Markdown
        converter = DocumentConverter()
        result = converter.convert(pdf_path)
        markdown_content = result.document.export_to_markdown()
        print(f"C: {filename}")

        all_markdown_content += " " + markdown_content

# Split the combined Markdown into chunks based on markdown headers.
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

chunked_data = text_splitter.split_text(all_markdown_content)
print("Len:", len(chunked_data))
chunked_data

C: wren_and_martin_simplified_textbook-1.pdf
C: wren_and_martin_simplified_textbook-2.pdf
C: wren_and_martin_simplified_textbook-3.pdf
C: wren_and_martin_simplified_textbook-4.pdf
C: wren_and_martin_simplified_textbook-5.pdf
C: wren_and_martin_simplified_textbook-6.pdf
C: wren_and_martin_simplified_textbook-7.pdf
C: wren_and_martin_simplified_textbook-8.pdf
C: wren_and_martin_simplified_textbook-9.pdf
C: wren_and_martin_simplified_textbook-10.pdf
C: wren_and_martin_simplified_textbook-11.pdf
C: wren_and_martin_simplified_textbook-12.pdf
C: wren_and_martin_simplified_textbook-13.pdf
C: wren_and_martin_simplified_textbook-14.pdf
C: wren_and_martin_simplified_textbook-15.pdf
C: wren_and_martin_simplified_textbook-16.pdf
C: wren_and_martin_simplified_textbook-17.pdf
C: wren_and_martin_simplified_textbook-18.pdf
C: wren_and_martin_simplified_textbook-19.pdf
C: wren_and_martin_simplified_textbook-20.pdf
C: wren_and_martin_simplified_textbook-21.pdf
C: wren_and_martin_simplified_textbook-22.p

[Document(metadata={}, page_content='NEW EDITION HIGH SCHOOL'),
 Document(metadata={'Header 2': 'English Grammar &amp; Composition'}, page_content='BY WREN &amp; MARTIN  \n(With New Appendices)  \nREVISED BY  \nN.D.V. PRASADA RAO  \nS. CHAND  \nPage i  \nNew Edition'),
 Document(metadata={'Header 2': 'HIGH SCHOOL ENGLISH GRAMMAR AND COMPOSITION'}, page_content="By  \nP.C. WREN, MA. (OXON) and H. MARTIN, M.A. (OXON), O.B.E.  \nRevised By  \nN.D.V. PRASADA RAO, M.A., D.T.E., Ph.D.  \nDear Students,  \nBeware of fake/pirated editions. Many of our best selling titles have been unlawfully  printed by unscrupulous persons. Your sincere effort in this direction may stop piracy and  save intellectuals' rights.  \nFor the genuine book check the 3-D hologram which gives a rainbow effect.  S. CHAND AN ISO 9001: 2000 COMPANY  \nS. CHAND &amp; COMPANY LTD. RAM NAGAR, NEW DELHI -110 055  \nPage iii"),
 Document(metadata={'Header 2': 'PREFACE TO THE NEW EDITION'}, page_content="Wren and Martin's monu

In [ ]:
import pickle

# with open("langchain_documents.pkl", "wb") as f:
#     pickle.dump(chunked_data, f)

with open("langchain_documents.pkl", "rb") as f:
    loaded_documents = pickle.load(f)

loaded_documents

# Langchain

In [ ]:
!uv pip install --system langchain langchain-openai faiss-cpu tiktoken langchain-google-genai google-generativeai

Using Python 3.11.11 environment at: /usr
Audited 6 packages in 82ms


In [ ]:
from google.colab import userdata
from operator import itemgetter
import os, pickle
from pprint import pprint
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda , RunnablePassthrough
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.llms import Ollama
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

chat_history = []

In [ ]:
vectorstore = FAISS.from_documents(
    loaded_documents, embedding=GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)
vectorstore.save_local("dp_pbl_vectorstore")

In [ ]:
# from azure.storage.blob import BlobServiceClient

# CONN_STR = str(userdata.get("AZURE_STORAGE_CONNECTION_STRING"))

# blob_service_client = BlobServiceClient.from_connection_string(conn_str=CONN_STR)
# container_client = blob_service_client.get_container_client("ncert-extraction-storage")
# print("\nListing blobs...")

# # List the blobs in the container
# blob_list = container_client.list_blobs()
# for blob in blob_list:
#     print("\t" + blob.name)

# try:
#     # Create the local vectorstore folder (if it doesn't exist)
#     os.makedirs("vectorstore", exist_ok=True)

#     for blob in container_client.list_blobs(prefix="NCERT_IX_C1/vectorstore/"):
#         # Extract the filename from the blob name (assuming simple structure)
#         filename = blob.name.split("/")[-1]
#         local_path = os.path.join("vectorstore", filename)  # Path within "vectorstore" folder

#         blob_client = container_client.get_blob_client(blob.name)
#         with open(local_path, "wb") as download_file:
#             download_file.write(blob_client.download_blob().readall())
# except Exception as ex:
#   # Handle download exceptions
#   print(f"Error downloading vectorstore files: {ex}")

In [ ]:
vectorstore = FAISS.load_local("dp_pbl_vectorstore", GoogleGenerativeAIEmbeddings(model="models/text-embedding-004"), allow_dangerous_deserialization=True)

retriever = vectorstore.as_retriever()

human_template = """Context: {context}
Question: {question}
"""
system_prompt = """
As a helpful and cheerful assistant dedicated to teaching grammar to students, your role is to strictly answer grammar-related queries. If a student makes a grammatical error in their query, you will correct it and provide an explanation of what is incorrect.
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
   [system_message_prompt, human_message_prompt]
)

pprint(chat_prompt)

model = ChatOpenAI()

# Chain
chain = (
   {
       "context": itemgetter("question") | retriever,  # Context from retriever
       "question": itemgetter("question"),
   }
   | chat_prompt
   | model
   | StrOutputParser()
)

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\nAs a helpful and cheerful assistant dedicated to teaching grammar to students, your role is to strictly answer grammar-related queries. If a student makes a grammatical error in their query, you will correct it and provide an explanation of what is incorrect.\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Context: {context}\nQuestion: {question}\n'))])


In [ ]:
vectorstore.similarity_search("Explain 2nd question in fill the blank spaces with nearest or next")

[Document(page_content="(c) Fill the blank spaces with 'oldest' and 'eldest' :-  \n1\\. Rustam is the --- of my uncle's five sons.  \n2\\. He is the --- member of the School Committee.  \n3\\. That is Antonio, the duke's --- son.  \n4\\. The --- mosque in the town is near the railway station.  \n5\\. Mr. Smith is the --- teacher in the school.  \n(d) Fill the blank spaces with 'farther' or 'further' :-  \n1\\. I can't walk any ---.  \n2\\. No --- reasons were given.  \n3\\. He walked off without --- ceremony.  \n4\\. Until --- orders Mr. K.S. Dave will act as Headmaster of Nira High School. 5. To let, a bungalow at Ridge Road. For --- particulars apply to Box. No. 65. Page 29  \n(e) Fill the blank spaces with 'latest' or 'last' :-  \n1\\. The --- news from China is very disquieting.  \n2\\. The --- time I saw him, he was in high spirits.  \n3\\. To-day is the --- day for receiving lenders.  \n4\\. We expect to get the --- news in a few hours.  \n5\\. The --- Moghul Emperor came to an i

In [ ]:
query = "Explain 2nd question in fill the blank spaces with nearest or next and give the answer" # @param {type:"string"}
output = chain.invoke({"question": str(query)})
print(output)

The correction in your question is: "Explain the 2nd question in 'Fill the blank spaces with 'nearest' or 'next' and provide the answer."

Explanation: The ordinal number "2nd" should be written as "2nd" with the letters "nd" following the number "2" to indicate its position in a sequence.

Answer: The 2nd question in the 'Fill the blank spaces with 'nearest' or 'next' is "The pillar-box is --- to my house." The correct word to fill in the blank is "nearest."


In [ ]:
pprint(chat_history)

[HumanMessage(content='Can you explain the topic states of matter ?'),
 AIMessage(content="Of course! Let's talk about states of matter. Matter can exist in three main states: solid, liquid, and gas. \n\nSolid: In a solid, the particles are packed tightly together and vibrate in place. Think of a solid like a group of people standing shoulder to shoulder in a line and barely moving.\n\nLiquid: In a liquid, the particles are close together but can move around each other. Imagine a liquid like a group of people mingling and moving around at a party.\n\nGas: In a gas, the particles are far apart and move freely. Picture a gas like a group of people spreading out and freely moving around in a large open space.\n\nDo you have any examples of solids, liquids, and gases that you can think of in your daily life? Let's discuss further to see if you can identify more examples!")]


# Ollama

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


ConnectError: [Errno 111] Connection refused

In [ ]:
!ollama serve &

time=2024-04-11T17:29:55.146Z level=INFO source=images.go:804 msg="total blobs: 0"
time=2024-04-11T17:29:55.147Z level=INFO source=images.go:811 msg="total unused blobs removed: 0"
time=2024-04-11T17:29:55.147Z level=INFO source=routes.go:1118 msg="Listening on 127.0.0.1:11434 (version 0.1.31)"
time=2024-04-11T17:29:55.148Z level=INFO source=payload_common.go:113 msg="Extracting dynamic libraries to /tmp/ollama2035018923/runners ..."
time=2024-04-11T17:30:01.531Z level=INFO source=payload_common.go:140 msg="Dynamic LLM libraries [cpu_avx2 rocm_v60000 cuda_v11 cpu cpu_avx]"
time=2024-04-11T17:30:01.531Z level=INFO source=gpu.go:115 msg="Detecting GPU type"
time=2024-04-11T17:30:01.531Z level=INFO source=gpu.go:265 msg="Searching for GPU management library libcudart.so*"
time=2024-04-11T17:30:01.536Z level=INFO source=gpu.go:311 msg="Discovered GPU libraries: [/tmp/ollama2035018923/runners/cuda_v11/libcudart.so.11.0 /usr/local/cuda/lib64/libcudart.so.12.2.140]"
time=2024-04-11T17:30:01.5

In [ ]:
import ollama
ollama.pull("phi")